In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
#!pip install scikit-learn==1.5.0
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 44.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
#!pip list

In [ ]:
df = pd.read_csv('merged.csv')  # Ersetzen Sie 'data.csv' durch den Pfad Ihrer Datei

# Features und Labels extrahieren
features = df[['x', 'z', 'yaw', 'pitch', 'roll']].values
labels = df['action'].values

# Labels encodieren
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Feature-Skalierung
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(scaled_features, encoded_labels, test_size=0.2, random_state=42)

# Daten in Tensoren umwandeln
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader erstellen
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# LSTM-Modell definieren
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
          h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
          c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
# Hyperparameter
input_size = 5  # Anzahl der Features
hidden_size = 128
output_size = len(label_encoder.classes_)
num_layers = 2
num_epochs = 500
learning_rate = 0.001

# Modell, Verlustfunktion und Optimierer instanziieren
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training des Modells
model.train()
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.unsqueeze(1)  # Batch-Dimension hinzufügen für LSTM
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/500], Loss: 0.6330
Epoch [2/500], Loss: 0.2565
Epoch [3/500], Loss: 0.5832
Epoch [4/500], Loss: 0.0257
Epoch [5/500], Loss: 0.2766
Epoch [6/500], Loss: 0.2362
Epoch [7/500], Loss: 0.0493
Epoch [8/500], Loss: 0.1366
Epoch [9/500], Loss: 0.0806
Epoch [10/500], Loss: 0.1779
Epoch [11/500], Loss: 0.0682
Epoch [12/500], Loss: 0.1717
Epoch [13/500], Loss: 0.0002
Epoch [14/500], Loss: 0.0132
Epoch [15/500], Loss: 0.2599
Epoch [16/500], Loss: 0.0062
Epoch [17/500], Loss: 0.2238
Epoch [18/500], Loss: 0.1184
Epoch [19/500], Loss: 0.1689
Epoch [20/500], Loss: 0.0185
Epoch [21/500], Loss: 0.0544
Epoch [22/500], Loss: 0.0531
Epoch [23/500], Loss: 0.0229
Epoch [24/500], Loss: 0.0849
Epoch [25/500], Loss: 0.0295
Epoch [26/500], Loss: 0.0533
Epoch [27/500], Loss: 0.0010
Epoch [28/500], Loss: 0.0117
Epoch [29/500], Loss: 0.0195
Epoch [30/500], Loss: 0.0921
Epoch [31/500], Loss: 0.0000
Epoch [32/500], Loss: 0.0175
Epoch [33/500], Loss: 0.0387
Epoch [34/500], Loss: 0.0001
Epoch [35/500], Loss: 0

In [ ]:
# Save the trained model
import pickle
torch.save(model.state_dict(), 'gesture_model.pth')
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

with open('label_encoder.pkl','wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
# Modell bewerten
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.unsqueeze(1)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 96.93%


In [ ]:
# Testen mit neuen Daten
def predict_new_data(new_data, model, scaler, label_encoder):
    # Neue Daten skalieren
    new_data_scaled = scaler.transform(new_data)

    # In Tensor umwandeln
    new_data_tensor = torch.tensor(new_data_scaled, dtype=torch.float32).unsqueeze(1)  # Batch-Dimension hinzufügen

    # Modell in den Evaluierungsmodus setzen
    model.eval()

    with torch.no_grad():
        outputs = model(new_data_tensor)
        _, predicted = torch.max(outputs.data, 1)

    # Vorhersagen decodieren
    predicted_labels = label_encoder.inverse_transform(predicted.cpu().numpy())

    return predicted_labels

In [ ]:
# Laden und Verfeinern des Modells
# Modell neu instanziieren und Zustand laden
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)
model.load_state_dict(torch.load('gesture_model.pth'))

# Optional: Modell weiter trainieren (feinabstimmen)
model.train()
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.unsqueeze(1)  # Batch-Dimension hinzufügen für LSTM
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Fine-tuning Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Speichern Sie das verfeinerte Modell
torch.save(model.state_dict(), 'refined_gesture_model.pth')

Fine-tuning Epoch [1/500], Loss: 0.0000
Fine-tuning Epoch [2/500], Loss: 0.0000
Fine-tuning Epoch [3/500], Loss: 0.0000
Fine-tuning Epoch [4/500], Loss: 0.0000
Fine-tuning Epoch [5/500], Loss: 0.0000
Fine-tuning Epoch [6/500], Loss: 0.0000
Fine-tuning Epoch [7/500], Loss: 0.0000
Fine-tuning Epoch [8/500], Loss: 0.0000
Fine-tuning Epoch [9/500], Loss: 0.0000
Fine-tuning Epoch [10/500], Loss: 0.0000
Fine-tuning Epoch [11/500], Loss: 0.0000
Fine-tuning Epoch [12/500], Loss: 0.0000
Fine-tuning Epoch [13/500], Loss: 0.0000
Fine-tuning Epoch [14/500], Loss: 0.0001
Fine-tuning Epoch [15/500], Loss: 0.0000
Fine-tuning Epoch [16/500], Loss: 0.0000
Fine-tuning Epoch [17/500], Loss: 0.0000
Fine-tuning Epoch [18/500], Loss: 0.0000
Fine-tuning Epoch [19/500], Loss: 0.0000
Fine-tuning Epoch [20/500], Loss: 0.0000
Fine-tuning Epoch [21/500], Loss: 0.0000
Fine-tuning Epoch [22/500], Loss: 0.0000
Fine-tuning Epoch [23/500], Loss: 0.0000
Fine-tuning Epoch [24/500], Loss: 0.0000
Fine-tuning Epoch [25/500